In [1]:
import faiss

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("../db")))
from sttcastdb import SttcastDB
sys.path.append(os.path.abspath("../tools"))
from envvars import load_env_vars_from_directory
import numpy as np

In [2]:
load_env_vars_from_directory("../.env")
OLD_FAISS_FILE = '/home/jmrobles/Podcasts/Coffee Break/DB/int_index.faiss'
NEW_FAISS_FILE = '/home/jmrobles/Podcasts/Coffee Break/DB/int_index_v2.faiss'
db_file = os.getenv('STTCAST_DB_FILE')
db_file

'/home/jmrobles/Podcasts/Coffee Break/DB/coffeebreak.db'

In [3]:
db  = SttcastDB(db_file)

In [5]:
ints = db.get_ints(with_embeddings=True)
print(len(ints))
for i in ints[-5:]:
    print(dict(i))

209956
{'id': 238825, 'start': 8481.64, 'end': 8499.05, 'epname': 'ep529_B', 'epdate': datetime.date(2025, 10, 23), 'tag': 'Héctor Socas', 'embedding': b'\xa8@\x08=b\x1f\x0e=\x0e\x03\x00\xbc\xf5\x9e\x18\xbc^_\xc1<\xd5H\n=`t\xb8<\x8a\xd1\xe4<\x80\xe1\x8c\xbc\xf7h \xbd\xf4\xbdJ\xbb\x02\x855\xbbW\x8a\xfd9\x88\xd33\xbd&\xd1\x08<\xaa\xf3\xc9<\xa4\xb4d\xbd,\xf9\'\xbd\x8b\xfd!<=\xbeC=\x18U\r\xbd\xc7jY;\xf0\xf5\x91<\x0e\x03\x00=B~\x90\xbc$q\xa2;}\xebG\xbd@\xb4\x88=W\xd5\xec<\x88\xd3\xb3\xbc\xa7Ht=h\xb1N\xbd\xc9\xb3y=\x03\xd0\xa4\xbc`\xa8a\xbd\xb7\x86\x8b\xbc!(\x82<\xd9\xa6\xa1\xbcS\x8e\x9b<x\x8d0\xbd\xb4\x90\xc6:\xbdz\x01\xbda\'z<\x01\xef\xd6<\x82\xdf=\xbc\xde\x049\xbd\x0e\xec9\xbd\x97d\xa6\xbc\x91<\x87=\x13\xff\xe1<#\xf2\x89\xbd\x91\xf1\x97\xbc\xcd\xf4-<\x17\x8b\x05>\xc6\xd4z\xbcl\xdb\xbc\xbcu\xae\xb1<\xd8\xc5\xd3<\x07.<<P\xcc\xff\xbc\xa9\xbf\xa0\xba|\xb7\x1e9d\x87`=\xfb\x92\x0e=$Z\\<\x83^V<\xf6\x1d\xb1\xbd\x9e\x8cE=\xd7]\x01\xbd\x8e]\x08=wY\x07\xbd\xfb\xc6\xb7<\xa1 U\xbd\xd4\x9b\xe59\xb8\xa3

In [6]:

embeddings = [
    np.frombuffer(dict(intv).get("embedding"), dtype=np.float32)
    for intv in ints
]
if embeddings is None or len(embeddings) == 0:
    raise ValueError("No embeddings found in the database.")
dimension = len(embeddings[0])
print(f"Dimension of embeddings: {dimension}")
print(f"Number of embeddings: {len(embeddings)}")

Dimension of embeddings: 1536
Number of embeddings: 209956


In [11]:
ids = [intv["id"] for intv in ints]
print(f"{len(ints)} - {len(ids)} - {
    len(embeddings)}")
ids[:10]

201601 - 201601 - 201601


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [23]:
index = faiss.IndexIDMap(faiss.IndexFlatL2(dimension))


In [24]:
index.add_with_ids(np.array(embeddings, dtype=np.float32), np.array(ids, dtype=np.int64))

In [25]:
faiss.write_index(index, NEW_FAISS_FILE)
print(f"Index saved to {NEW_FAISS_FILE}")

Index saved to /home/jmrobles/Podcasts/Coffee Break/DB/int_index_v2.faiss
